# Imports

In [ ]:
from despliegue.loaders import *
from despliegue.nodos import *
from despliegue.contenedores import *
from despliegue.solvers import *
import pandas as pd
from os import path

# Constantes

Se definen las rutas de los datos, y también el `DataFrame` de la Red Móvil, ya que no posee la vacancia, y hay que asignarla a mano.

In [ ]:
PATH_DATA = path.join("./data/")  # Path donde estarán los Excels
PATH_CLIENTES = path.join(PATH_DATA, "Direcciones_Colina.xlsx")  # Path de los clientes
PATH_FO = path.join(PATH_DATA, "CTO_Colina.xlsx")  # Path donde estará las CTO's
PATH_RM = path.join(PATH_DATA, "SITIOS 4G TDD.xlsx")  # Path donde estarán las antenas de RM

In [ ]:
df_clientes = pd.read_excel(PATH_CLIENTES)
df_clientes = df_clientes[df_clientes.gl_lat_OK <= -1]  # limpiamos aquellos que sean de lat-lon=0
df_clientes.describe()

In [ ]:
df_fo = pd.read_excel(PATH_FO)
df_fo.describe()

In [ ]:
df_rm = pd.read_excel(PATH_RM)
df_rm["vacancia"] = 30
df_rm.describe()

In [ ]:
df_rm.head()

# Conexión a los Datos

Creamos las instancias de conexión a la base de datos. Observar que para `rm_db` le entregamos el `DataFrame` directamente.

In [ ]:
cliente_db = ClienteDB(df=df_clientes, col_names=["pcm_area_tel", "gl_lat_OK", "gl_lon_OK"])
fo_db = OfertaDB(df=df_fo, col_names=["id_pto_ftth", "latitud_ok", "longitud_ok", "eqpt_vg_qty"])
rm_db = OfertaDB(df=df_rm)  # Las columnas justamente están ordenadas en [id, lat, lon, vac]

# Conjuntos de Nodos

Creamos los conjuntos de nodos utilizando los loaders de la parte anterior

In [ ]:
nodos_oferta = NodosOferta(fo_db, rm_db)
nodos_demanda = NodosDemanda(cliente_db)

# Optimización

Creamos una instancia del tercer solver, que se corresponde con el siguiente modelo:
\begin{equation}
\left\{\begin{array}{rll}
\max & \sum_{i=1}^{n} \sum_{j\in N(i)} c_{ij} x_{ij} & \\
\mbox{s.a.} & \sum_{j\in N(i)} x_{ij} \leq O_i.vac & \forall i = 1:n \\
& \sum_{i \in N(j)} x_{ij} = 1 & \forall j = 1 : m \\
& x_{ij} \in \{0, 1\} &
\end{array}\right.
\end{equation}

Donde $N(i)$ son los vecinos del vértice $i$. Y consideramos a las aristas 
$$E = \{
ij \ | \ i \in FO, \ j=1:m, \ O_i.\mbox{dist}_1(D_j) \leq C_{FO}
\} \cup \{
ij \ | \ i \in RM, \ j=1:m, \ O_i.\mbox{dist}_2(D_j) \leq C_{RM}
\}$$

Además, interpretamos a la matriz $(c_{ij})_{ij}$ como una matriz de prioridad, donde se le dará mayor prioridad a la arista $ij$ en tanto tenga mayor $c_{ij}$. Consideramos la siguiente matriz:
\begin{equation}
c_{ij}
= \begin{cases}
\frac{a}{O_i.\mbox{dist}_1(D_j) + \varepsilon} & i \in FO \\
\frac{b}{O_i.\mbox{dist}_2(D_j) + \varepsilon} & i \in RM \\
0 & \mbox{en cualquier otro caso}
\end{cases}
\end{equation}

Intuitivamente, lo que hace es darle mayor prioridad a aquellos clientes que estén más cerca a cierta CTO o RM. 

In [ ]:
modelo3 = Solver3(nodos_oferta, nodos_demanda, a=2, b=1, c_fo=150, c_rm=600, verbose=True)
modelo3.resolver()

In [ ]:
modelo3.save(path.join(PATH_DATA, "resultado.xlsx"))

Observamos que considerar este modelo reduce de casi un millón de variables a casi 4500, aumentando mucho la eficiencia en el tiempo.

In [ ]:
modelo1 = Solver1(nodos_oferta, nodos_demanda)
modelo3 = Solver3(nodos_oferta, nodos_demanda)
len(modelo1.x), len(modelo3.x)